In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_squared_error
%matplotlib inline

# **EDA**

In [ ]:
df1 = pd.read_csv('../input/food-prices-in-turkey/train.csv')
df2 = pd.read_csv('../input/food-prices-in-turkey/test.csv')

In [ ]:
df1.shape, df2.shape

In [ ]:
df1.isna().sum()

In [ ]:
df2.isna().sum()

In [ ]:
df = pd.concat([df1, df2], axis=0)

In [ ]:
df.shape

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df.drop(['ProductId', 'UmId'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df['ProductName'].value_counts()

In [ ]:
cols_to_replace = df['ProductName'].value_counts()[df['ProductName'].value_counts() < 50].keys()

In [ ]:
df['ProductName'].replace(cols_to_replace,['Others' for i in range(6)],inplace=True)

In [ ]:
df['ProductName'].value_counts()['Others']

In [ ]:
plt.figure(figsize=(15,15))
df['ProductName'].value_counts().plot(kind='barh')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df.dtypes

In [ ]:
df.Place.value_counts().plot(kind='barh')
plt.gca().invert_yaxis()
plt.show()


In [ ]:
df.UmName.value_counts()

In [ ]:
df['Place'].value_counts().plot(kind='barh')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
df_dummies = pd.get_dummies(df[['Place', 'ProductName', 'UmName']],drop_first=True)

In [ ]:
df_dummies.columns

In [ ]:
df['Year'] = df['Year']/10000
df['Month'] = df['Month']/10000


In [ ]:
df.head()

# **One Hot Encoding**

In [ ]:
df_main = pd.concat([df_dummies, df[['Month', 'Year', 'Price']]], axis=1)

In [ ]:
df_main.shape

In [ ]:
df_main.head()

# **Splitting and Training**

In [ ]:
x, y = df_main.drop(['Price'],axis=1), df_main['Price']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
y_train.shape, y_test.shape

# **Model Build and Predictions**

In [ ]:
model = LinearRegression()
model.fit(x_train, y_train)

In [ ]:
y_pred_test = model.predict(x_test)
y_pred_train = model.predict(x_train)

In [ ]:
model.score(x_test, y_test)

In [ ]:
model.score(x_train, y_train)

In [ ]:
mean_squared_error(y_test, y_pred_test)

In [ ]:
mean_squared_error(y_train, y_pred_train)

# **Applying KNRegressor**

In [ ]:
model_kr = KNeighborsRegressor(n_neighbors=3,metric='euclidean',weights='distance')
model_kr.fit(x_train, y_train)

In [ ]:
model_kr.score(x_test, y_test)

In [ ]:
model_kr.score(x_train, y_train)

In [ ]:
clf = GridSearchCV(KNeighborsRegressor(),
                   param_grid={
                       'metric':['euclidean','minkowski','manhattan'],
                       'weights':['uniform','distance'],
                       'n_neighbors':[3,5,7,11]
                   },
                   cv=7
                  )

In [ ]:
clf.fit(x_train, y_train)

In [ ]:
cv_result = pd.DataFrame(clf.cv_results_)
cv_result

In [ ]:
cv_result[['param_weights','params','mean_test_score']]

# **Best Estimator and Model Prediction**

In [ ]:
clf.best_estimator_

In [ ]:
model_kr2 = KNeighborsRegressor(metric='manhattan', n_neighbors=3, weights='distance')
model_kr2.fit(x_train, y_train)

In [ ]:
model_kr2.score(x_test, y_test)

In [ ]:
model_kr2.score(x_train, y_train)

In [ ]:
y_pred_test_kr = model_kr2.predict(x_test)
y_pred_train_kr = model_kr2.predict(x_train)

In [ ]:
mean_squared_error(y_test, y_pred_test_kr)

In [ ]:
mean_squared_error(y_train, y_pred_train_kr)

In [ ]:
pred_kn_test = pd.DataFrame({'Y test KN':y_test,'Y predicted KN':y_pred_test_kr})
pred_kn_train = pd.DataFrame({'Y Train KN':y_train,'Y predicted KN':y_pred_train_kr})

# **Correlation Test and Predicted**

In [ ]:
pred_kn_test.corr()

In [ ]:
pred_kn_train.corr()

In [ ]:
pred_kn_test.sample(10)

In [ ]:
x_train.to_csv('MyTrain.csv',index=False)
x_test.to_csv('MyTest.csv',index=False)

In [ ]:
import os
os.listdir()